## Numerical Experiments

Simulation code for running the numerical experiments reported in the paper:  
**Vidit Saxena and Joakim Jaldén,"Bayesian Link Adaptation under a BLER Target", In 2020 IEEE 21st International Workshop on Signal Processing Advances in Wireless Communications (SPAWC) on May 26-29, 2020.** 

This simulation code is written in Python3. Running each of the subsequent cells in sequence will execute the experiments and generate a results file (in .npy format) that is saved to the disk.

The simulations make extensive use of the [py-itpp](https://github.com/vidits-kth/py-itpp), [Numpy](https://github.com/numpy/numpy), and [Matplotlib](https://github.com/matplotlib/matplotlib) packages.

Additionally, to speed up the generation of results, the simulations are parallezlized using the [Ray](http://ray.readthedocs.io/en/latest/index.html) package. It is possible to run single-threaded simulations at the cost of slowness, by commenting out the Ray-specific lines in the notebook - this is indicated in the appropriate sections of the code.

The user-defined parameters are highlighted with the USER-DEFINED tag below; set these values below to setup and run the experiments for the desired parameters.

### Import Dependencies

In [ ]:
from matplotlib import pyplot as plt

import numpy as np
import ray
import time

from source import simluate_rayleigh_fading_channel
from source import ThompsonSamplingBandit, OuterLoopLinkAdaptation

plt.rcParams.update({'font.size': 22, 'lines.linewidth' : 3})

# RAY: If Ray is installed on this machine, initialize it properly below. 
# It is also possible to run the experiments without Ray. In that case, comment out the lines below and in 
# subsequent cells, where identified with the RAY: comment.
ray_redis_address = "10.0.0.5:15672"
ray.init(address=ray_redis_address, ignore_reinit_error=True, log_to_driver=False)

### Load SNR-to-CQI Lookup Data
The CQI values are calculated using an offline lookup table of AWGN BLER values. For a complete description of the CQI generation technique, refer to [11] referenced in the pabler. This lookup data is used for AMC part of the AMC-OLLA technique.

The AWGN_DATASET is generated offline using a separate simulation (not reported here since it is not relevant to this pabler). This dataset contains the measured BLER for a large range of average SNRs, for each of the MCSs simulated in this pabler.

In [ ]:
awgn_datafile = 'AWGN_DATASET.npy'
awgn_data = np.load( awgn_datafile, allow_pickle=True )[ ( ) ]

snr_vs_bler = awgn_data['snr_vs_per']
snr_range_dB = awgn_data['snr_range_dB']

nrof_snr, nrof_rates = snr_vs_bler.shape

In [ ]:
# Visualize the lookup data
plt.figure(figsize=[20,5])

legend = []
for i in range( nrof_rates ):
    plt.semilogy( snr_range_dB, snr_vs_bler[:,i] )
    legend.append('MCS %d'%(i))
    
plt.legend(legend, ncol=2)
plt.title('SNR-vs-BLER data for CQI lookup')
plt.xlabel('Average SNR [dB]')
plt.ylabel('BLER')

### Simulation parameters

In [ ]:
# Standard-defined transport block sizes from [12, Tab. 7.1.7.2.1-1]
packet_sizes = [152, 200, 248, 320, 408, 504, 600, 712, 808, 936, 
                936, 1032, 1192, 1352, 1544, 1736, 1800, 
                1800, 1928, 2152, 2344, 2600, 2792, 2984, 3240, 3496, 3624, 3752, 4008]

# Standard-defined Modulation orders from [12, Tab. 7.1.7.1-1]
modorders    = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
                4, 4, 4, 4, 4, 4, 4, 
                6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

# USER-DEFINED target BLER and OLLA parameters
target_bler = 0.1
olla_step_size = 0.1

# USER-DEFINED wireless channel configuration
norm_doppler = 0.01
avg_snr_dB = 15

# USER-DEFINED exbleriment parameters
nrof_ttis = 50
nrof_experiments = 10

# Parameter settings for Figs. 1(a) and 1(b) in the paper:
# target_bler = 0.1, olla_step_size = 0.1, norm_doppler = 0.01, avg_snr_dB = 15, nrof_ttis = 5000, nrof_experiments = 1000

# Parameter settings for Figs. 1(a) and 1(b) in the paper:
# target_bler = 0.3, olla_step_size = 0.1, norm_doppler = 0.01, avg_snr_dB = 15, nrof_ttis = 5000, nrof_experiments = 1000

### Extract prior BLER corresponding to each CQI value

In [ ]:
from source import estimate_sinr_from_cqi, determine_bler_at_sinr

nrof_cqi = 16
bler_bler_cqi = np.ndarray( ( len( packet_sizes ), nrof_cqi ) )
for cqi in range( nrof_cqi ):
    snr_dB = estimate_sinr_from_cqi(cqi, awgn_data)
    bler_bler_cqi[ :, cqi ] = determine_bler_at_sinr(snr_dB, awgn_data)

### OLLA and BaysLA for link adaptation over a Rayleigh Fading channel

In [ ]:
# RAY: The following method is packaged using Ray. To remove this dependence, comment out the line below.
@ray.remote  # Comment this out if Ray is not used
def run_experiment( seed, nrof_ttis, avg_snr_dB, target_bler, olla_step_size, norm_doppler ):
    
    from source import simluate_rayleigh_fading_channel
    from source import ThompsonSamplingBandit, OuterLoopLinkAdaptation

    packet_error_probabilities, channel_quality_indices = simluate_rayleigh_fading_channel( nrof_ttis, 
                                                                                            avg_snr_dB, 
                                                                                            awgn_data, 
                                                                                            packet_sizes, 
                                                                                            norm_doppler, 
                                                                                            seed = seed )

    # Pre-generate ACK events for all rates for all channel samples
    packet_acks = np.ndarray( ( nrof_ttis, nrof_rates ) )
    for tti in range( nrof_ttis ):
        for rate_index in range( nrof_rates ):
            packet_acks[tti, rate_index] = np.random.uniform( ) > packet_error_probabilities[tti, rate_index]

            
    # Outer Loop Link Adaptation
    olla_bandit = OuterLoopLinkAdaptation(nrof_rates, packet_sizes, awgn_data, target_bler, olla_step_size)
    
    cqi_delay = 0 
    
    olla_rates  = []
    olla_acks  = []
    olla_tputs = []
    for tti in range( nrof_ttis ):
        
        # Skip the first few samples to account for CQI delay
        if tti < cqi_delay:
            selected_rate_index = np.random.randint(0, nrof_cqi)
            ack = packet_acks[tti, selected_rate_index]
        else:  
            cqi = channel_quality_indices[tti - cqi_delay]    
            selected_rate_index = olla_bandit.act( cqi )

            ack = packet_acks[tti, selected_rate_index]
            olla_bandit.update( selected_rate_index, cqi, ack )

        olla_rates.append(selected_rate_index)
        olla_acks.append(ack)
        olla_tputs.append( packet_sizes[ selected_rate_index ] * ack )
        
    
    # Thompson Sampling with Informed Priors
    bayesla_bandit = ThompsonSamplingBandit(nrof_rates, packet_sizes, target_bler, bler_bler_cqi)
    
    bayesla_rates  = []
    bayesla_acks  = []
    bayesla_tputs = []
    for tti in range( nrof_ttis ):
        
        # Skip the first few samples to account for CQI delay
        if tti < cqi_delay:
            selected_rate_index = np.random.randint(0, nrof_cqi)
            ack = packet_acks[tti, selected_rate_index]
        else:    
            cqi = channel_quality_indices[tti - cqi_delay]    
            selected_rate_index = bayesla_bandit.act( cqi )

            ack = packet_acks[tti, selected_rate_index]
            bayesla_bandit.update( selected_rate_index, cqi, ack )

        bayesla_rates.append(selected_rate_index)
        bayesla_acks.append(ack)
        bayesla_tputs.append( packet_sizes[ selected_rate_index ] * ack )
    
    return ( olla_rates, olla_acks, olla_tputs, 
             bayesla_rates, bayesla_acks, bayesla_tputs, )

### Run the experiments and collect results

In [ ]:
olla_mcs = []
olla_ack = []
olla_tput = []

bayesla_mcs = []
bayesla_ack = []
bayesla_tput = []


# RAY: The following line runs the Ray-packaged method. To avoid using Ray, comment out the next line and uncomment
# the line that immediately follows it in order to run the experiments.
results = ray.get( [ run_experiment.remote( i,
                                            nrof_ttis, 
                                            avg_snr_dB, 
                                            target_bler,
                                            olla_step_size,
                                            norm_doppler ) for i in range(nrof_experiments) ] )

# RAY: uncomment the next line to run a single-threaded simulation that does not depend on Ray.
#results = [ run_experiment( i, nrof_ttis, avg_snr_dB, target_bler, olla_step_size, norm_doppler ) for i in range(nrof_experiments) ]

### Extract the results in separate variables for OLLA and BayesLA

In [ ]:
for res in results:
    olla_mcs.append( res[0] )
    olla_ack.append( res[1] )
    olla_tput.append( res[2] )
    
    bayesla_mcs.append( res[3] )
    bayesla_ack.append( res[4] )
    bayesla_tput.append( res[5] )

### Save the results to disk

In [ ]:
data= { 'olla': ([], olla_ack, olla_tput),
        'bayesla': ([], bayesla_ack, bayesla_tput)}

In [ ]:
#result_file = 'RESULT_SNR_%d_TARGET_%0.1f_DELAY_%d.npy'%( avg_snr_dB, target_bler, cqi_delay)

result_file = 'TEST.npy'

np.save(result_file, data)

print('Saved to %s'%(result_file))